In [5]:
import openai

In [6]:
from openai import OpenAI

In [7]:
client = OpenAI()

In [11]:
response = client.chat.completions.create(
    model='gpt-4.1-nano',
    messages=[{"role":"user", "content":"is it too late to join the course?"}]
)

In [13]:
response.choices[0].message.content

"To give you the best answer, could you please specify which course you're referring to?"